### AI Hand detection (chinese number hand Gestures)

In [1]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp

In [2]:
# Drawing helpers
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles
# Mediapipe Solution for hands
mp_hands = mp.solutions.hands 

In [112]:
cap = cv2.VideoCapture(0)
# Initiate Hand model 
with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.8) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # Color the image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        # Recolor the image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw hands landmarks
        #print(results.multi_hand_landmarks)
        if results.multi_hand_landmarks:
            for num, hand_landmarks in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand_landmarks,mp_hands.HAND_CONNECTIONS, 
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        #cv2.imshow('Raw webcam Feed', image)
        #if cv2.waitKey(5) & 0xFF == 27:
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()  

### 2. Capture Landmarks & Export to CSV

In [12]:
import csv
import os

In [13]:
len(hand_landmarks.landmark)

21

In [19]:
# get the number of coordinates hand landmarkes
landmarks = ['class']
for val in range(1,len(hand_landmarks.landmark)+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]
#landmarks += [f"'x{val}', 'y{val}', 'z{val}', 'v{val}'"]
landmarks[-1]

'z21'

In [20]:
#landmarks.to_csv('coords.csv')
# Save the coordination into a csv file
with open('hands_coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [21]:
d = pd.read_csv('hands_coords.csv')
d

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z18,x19,y19,z19,x20,y20,z20,x21,y21,z21


In [45]:
#class_name ='One'
#class_name ='Two'
#class_name ='Three'
#class_name ='Four'
#class_name = 'Five'
#class_name = 'Six'
#class_name = 'Seven'
#class_name = 'Eight'
#class_name = 'Nine'
class_name = 'Ten'

In [46]:
cap = cv2.VideoCapture(0)
# Initiate Hand model 
with mp_hands.Hands(model_complexity=1,min_detection_confidence=0.8,min_tracking_confidence=0.8) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # Color the image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        # Recolor the image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw hands landmarks
        #print(results.multi_hand_landmarks)
        if results.multi_hand_landmarks:
            for num,hand_landmarks in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand_landmarks,mp_hands.HAND_CONNECTIONS, 
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
                
                #print(hand_landmarks)
                try: 
                    # Extracting hands landmarks
                    hand = hand_landmarks.landmark
                    #print(len(hand))
                    hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())
                    #print(hand_row)
               
                    # Append class name
                    hand_row.insert(0, class_name)
                
                    # Export to csv
                    with open('hands_coords.csv', mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        csv_writer.writerow(hand_row)
                except:
                     pass
        
        
        
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('Hands Detection Chinese number signe', cv2.flip(image, 1))
        #cv2.imshow('Raw webcam Feed', image)
        
        #if cv2.waitKey(5) & 0xFF == 27:
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()  

In [27]:
#d = pd.read_csv('hands_coords.csv')
#d

### 3. Train Custom Model Using Scikit Learn

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [85]:
# Read the data 
df = pd.read_csv('hands_coords.csv')

In [86]:
df

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z18,x19,y19,z19,x20,y20,z20,x21,y21,z21
0,One,0.416197,0.863851,3.830000e-07,0.449296,0.831171,-0.024477,0.482840,0.719790,-0.037764,...,-0.042551,0.328088,0.591670,-0.071248,0.364177,0.621985,-0.068236,0.384958,0.669497,-0.059490
1,One,0.402853,0.876655,3.090000e-07,0.442705,0.826288,-0.022636,0.471480,0.706888,-0.035855,...,-0.036171,0.315156,0.563996,-0.064009,0.352767,0.582632,-0.064124,0.376748,0.628437,-0.057427
2,One,0.396258,0.891994,2.230000e-07,0.438062,0.822056,-0.019382,0.464252,0.694281,-0.030401,...,-0.033788,0.309911,0.547095,-0.060103,0.346400,0.565829,-0.059407,0.368012,0.609659,-0.052388
3,One,0.385525,0.905640,1.850000e-07,0.432279,0.829213,-0.018167,0.459544,0.701119,-0.028401,...,-0.037407,0.299560,0.547196,-0.064280,0.335703,0.567032,-0.065634,0.359775,0.616459,-0.060704
4,One,0.375671,0.916165,2.310000e-07,0.420435,0.837883,-0.021807,0.449157,0.710833,-0.035822,...,-0.049245,0.274537,0.566971,-0.076899,0.313079,0.582298,-0.076283,0.340248,0.625536,-0.069794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2696,Ten,0.302148,0.887597,6.280000e-07,0.359051,0.838923,-0.026713,0.399651,0.725403,-0.041589,...,-0.060141,0.281481,0.610800,-0.103667,0.313324,0.675776,-0.111981,0.328730,0.725672,-0.108838
2697,Ten,0.298238,0.887160,6.470000e-07,0.357721,0.837102,-0.027348,0.399147,0.721578,-0.042341,...,-0.063212,0.282513,0.606704,-0.108489,0.315469,0.674252,-0.116812,0.331617,0.726140,-0.113630
2698,Ten,0.301109,0.884700,6.290000e-07,0.357890,0.834802,-0.024559,0.397746,0.719664,-0.037750,...,-0.059826,0.275343,0.598769,-0.102376,0.307094,0.662406,-0.108547,0.322594,0.712675,-0.103846
2699,Ten,0.295848,0.884661,6.080000e-07,0.353980,0.833396,-0.023670,0.394796,0.719867,-0.036030,...,-0.056467,0.272412,0.593539,-0.100735,0.305536,0.656094,-0.108369,0.323281,0.708694,-0.104966


In [87]:
# Set features and Target variabes
X = df.drop('class', axis=1) # features
y = df['class'] # target
y

0       One
1       One
2       One
3       One
4       One
       ... 
2696    Ten
2697    Ten
2698    Ten
2699    Ten
2700    Ten
Name: class, Length: 2701, dtype: object

In [88]:
# Splite the data intro testing amd training
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1234)

In [89]:
#3.2 Train Machine Learning Classification Model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [90]:
# create a pipe line for the differents models
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [91]:
# Train the models 
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [92]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [93]:
# Test our model
fit_models['rf'].predict(X_test)

array(['Six', 'One', 'Two', 'Nine', 'Ten', 'Eight', 'Seven', 'Seven',
       'One', 'Nine', 'Six', 'Two', 'Five', 'Nine', 'Two', 'Five', 'Five',
       'Ten', 'Three', 'Four', 'Ten', 'Seven', 'Three', 'Nine', 'Three',
       'Six', 'Seven', 'Four', 'Four', 'One', 'Two', 'Nine', 'Four',
       'Six', 'Nine', 'Six', 'Nine', 'Two', 'One', 'Six', 'Ten', 'Eight',
       'Three', 'Five', 'Two', 'Seven', 'Eight', 'Nine', 'Three', 'Six',
       'Four', 'One', 'Ten', 'Two', 'Two', 'Eight', 'Five', 'Eight',
       'Six', 'Five', 'Seven', 'Seven', 'Three', 'Nine', 'One', 'Nine',
       'Ten', 'Eight', 'Three', 'Two', 'Two', 'Seven', 'Two', 'One',
       'Seven', 'One', 'Four', 'Five', 'Three', 'Two', 'Seven', 'Five',
       'One', 'Two', 'Four', 'Two', 'Nine', 'Two', 'Eight', 'Five',
       'Three', 'Seven', 'Seven', 'Two', 'Two', 'Four', 'Seven', 'Nine',
       'Ten', 'One', 'Nine', 'Two', 'Nine', 'Eight', 'Six', 'Nine',
       'Seven', 'Four', 'Nine', 'Four', 'One', 'Three', 'One', 'Six',
     

### 3.3 Evaluate and Serialize Model

In [94]:
from sklearn.metrics import accuracy_score
import pickle

In [95]:
# GET the evaluation of each model
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9901356350184957
rc 0.9889025893958077
rf 0.9889025893958077
gb 0.9802712700369913


In [108]:
# Let's save the model with pickle.
with open ('hand_gestures.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

In [109]:
# import the model
with open('hand_gestures.pkl', 'rb') as f:
    model = pickle.load(f)

In [110]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [114]:
cap = cv2.VideoCapture(0)
# Initiate Hand model 
with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.8) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        # Color the image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        # Recolor the image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw hands landmarks
        #print(results.multi_hand_landmarks)
        if results.multi_hand_landmarks:
            for num,hand_landmarks in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand_landmarks,mp_hands.HAND_CONNECTIONS, 
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
                
                #print(hand_landmarks)
                
                # Extracting hands landmarks
                hand = hand_landmarks.landmark
                #print(len(hand))
                hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())
                #print(hand_row)
               
                
                
            # Make Detection
            X = pd.DataFrame([hand_row])
            number_language_class = model.predict(X)[0]
            number_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            
            #Grab ear coords
            coords =  tuple(np.multiply(
                            np.array(
                                    (hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x,
                                    hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y)),
                           [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(number_language_class)*20, coords[1]-30),
                         (245,117,16), -1)
            cv2.putText(image, number_language_class, coords,
                        cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get the status box
            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)
            
            max_prob = round(number_language_prob[np.argmax(number_language_prob)],2)
            if max_prob > 0.6:
            
                # Display class
                cv2.putText(image, 'CLASS', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0),1, cv2.LINE_AA)
                cv2.putText(image, number_language_class.split(' ')[0], (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2, cv2.LINE_AA)
            
                # Display probability
            
            
                cv2.putText(image, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0),1, cv2.LINE_AA)
                cv2.putText(image, str(max_prob), 
                        (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2, cv2.LINE_AA)
            
            
        #cv2.imshow('Raw webcam Feed', image)
        cv2.imshow('Hands Detection Chinese number signe', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()  

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

...

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use

/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Users/gakas/AI_Body_Language_Decoder/body_language_ven/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/Use